Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
from tqdm import tqdm
import numpy

import lab9_lib
from GA import Ga, Individual
from ES import Es, Hillclimber

Toy example to show how it works

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    # ind = [1] * 50
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11000110100000110101011110110101001101001100000000: 7.33%
00100011100100110101110010110101101011110001001000: 15.33%
00001111001110000011100011010010001110010000001011: 29.56%
01111111101111110000110000100010011000100011110111: 15.33%
01001011010101000100101101011101000000101010010001: 17.56%
10100001101101111000011001111010011010110111101011: 15.33%
10100101010010100011010011010000010110011010101100: 29.56%
11010110110110011110111110001100010010111111011110: 9.11%
00001010101011010111100001011111111101100000001000: 7.33%
11000100010101011000101011110100101100110001001000: 7.34%
10


## Using ES
the provided parameters are working in most of the cases

In [38]:
sizes = [1, 2, 5, 10]
N_MUTATIONS = 100                                   # initial number of mutating genes
SIGMA = 1                        
PM = 0.9                                            # P to prefer mutation over xover
PARAMETER_FREEZE_THRESHOLD = 0.9                    # fitness threshold after which the basic parameters are frozen (stop exploring)
                                                    #  values >= 1 make the exploration go on 

POPULATION_SIZE = 5 if PM != 1 else 1               # can't do xover if only 1 guy is present, no? 
OFFSPRING_SIZE = 20

for problem_size in sizes:
    fitness = lab9_lib.make_problem(problem_size)
    #initialize population
    first_ones = [Hillclimber(numpy.array(choices([0, 1], k=1000)), fitness, N_MUTATIONS, SIGMA, PARAMETER_FREEZE_THRESHOLD) for _ in range(POPULATION_SIZE)]
    population = Es(POPULATION_SIZE, OFFSPRING_SIZE, first_ones, PM)
    
    # usually converges way before, but it was put to limit the number of evaluations
    for _ in range(100):
        if population.generate_offspring() != False:
            break
    print(f"{problem_size} -> {population.population[0].fitness:.2%}, used {fitness.calls} calls")       
    print('sigma:', population.population[0].sigma)
    print('nloci:', population.population[0].nloci)
    print('p1: ', population.population[0].p1, '\n\n')
   


1 -> 100.00%, used 65 calls
sigma: 0.9702605052368167
nloci: 1200
p1:  1 


2 -> 100.00%, used 65 calls
sigma: 0.9366349541677955
nloci: 1200
p1:  1 


5 -> 100.00%, used 45 calls
sigma: 1.0096446759489974
nloci: 900
p1:  1 


10 -> 100.00%, used 305 calls
sigma: 0.9759283578258094
nloci: 1600
p1:  1 




## We try to solve the problem using GA
it does not work, i kept the code if you want to play around

In [30]:
sizes = [1, 2, 5, 10]
k = 1000

POPULATION_SIZE = 10
OFFSPRING_SIZE = 50
N = 1
PM = 0.2
TOURNAMENT_SIZE = 2

for problem_size in sizes:
    fitness = lab9_lib.make_problem(problem_size)

    population = Ga(POPULATION_SIZE, fitness, N)
    for _ in range(100):
        #try solve the problem
        population.generate_offspring_1p(OFFSPRING_SIZE, PM, TOURNAMENT_SIZE)
        population.survival_selection(POPULATION_SIZE)
        if population.population[0].fitness == 1:
            break

    print(f"{problem_size} -> {population.population[0].fitness:.2%}, used {fitness.calls} calls\n")

1 -> 91.90%, used 30184 calls

2 -> 47.79%, used 31563 calls

5 -> 19.69%, used 31576 calls

10 -> 14.42%, used 33319 calls

